In [1]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import glob
from nltk.stem.porter import PorterStemmer
from itertools import chain #untuk flaten array 2d
from collections import defaultdict
from collections import OrderedDict

In [2]:
def readFile(directory):
    listFile = glob.glob(directory+"/*.txt")
    return listFile

## normalisasi

In [3]:
def normalisasi(body):

    #hapus tag
    hapusTag = r'<DATE>|</DATE>|<TITLE>|</TITLE>|<BODY>|</BODY>'
    body = re.sub(hapusTag, ' ', body)

    #hapus \n
    hapusEnter = r'\n'
    body = re.sub(hapusEnter, ' ', body)

    # hapus htmlEntities 
    htmlEntities = r"&lt;"
    body = re.sub(htmlEntities, '', body)

    kutip2 = r'[\"\(\)\<\>\+\?]'
    body = re.sub(kutip2, '', body)

    # hapus titik dan koma pada akhir string
    titikDiAkhirString = r"([a-zA-Z]+)([^\w\s])"
    temp = re.findall(titikDiAkhirString, body)
    x = True
    while x :
#         print(temp)
        temp = re.findall(titikDiAkhirString, body)
        if len(temp) == 0 :
            x = False
        for i in temp:
            temp2 = i[0] + i[1]
            body = re.sub(temp2, i[0], body)

    # hapus titik dan koma pada akhir angka    
    titikSetelahAngka = r'(\d+)(^\w)\s'    
    temp = re.findall(titikSetelahAngka,body)
    while x :
#         print(temp)
        temp = re.findall(titikSetelahAngka, body)
        if len(temp) == 0:
            x = False
        for i in temp:
            temp2 = i[0] + i[1]
            body = re.sub(temp2, i[0], body)

    kutips = r"'s"
    body = re.sub(kutips, '', body)
    
    spasiLebihDariSatu = r'\s+'
    body = re.sub(spasiLebihDariSatu, ' ', body)
    return body

In [4]:
def readText(file):
    words = []
    f = open(file, 'r') #open file
    text = f.read()    
    f.close()

    return text

In [5]:
def removeEnter(listOfWords): #['<BODY>\n']
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].replace('\n','')
        
    return listOfWords

In [6]:
def removeNull(listOfWords): #['','','eat','food','','']
    listOfWords = list(filter(None, listOfWords))
    
    return listOfWords

In [7]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

<h3>Load Stop Word from NLTK</h3>

In [8]:
stop_words = stopwords.words('english')

In [9]:
def removeStopWord(listOfWords):
    for i in listOfWords:
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

<h3>Load Lemmatizer from NLTK</h3>

In [10]:
lemmatizer = WordNetLemmatizer()

In [11]:
def lemmatization(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = lemmatizer.lemmatize(listOfWords[i])
        
    return listOfWords

<h3>Load Stemming from NLTK</h3>

In [12]:
def stemming(listOfWords):
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in listOfWords]
    
    return stemmed

# Token

In [13]:
def make_token(txt):
    list_hasil = txt.split(" ")            
    return list_hasil

# Term

In [14]:
def make_term(file_name_token,file_name_term):
    list_term = []
    f = open(file_name_token, 'r')
    text = f.read()
    f.close()
    
    list_token = text.split("\n")
    for i in list_token:
        if i not in list_term:
            list_term.append(i)
            
    f1 = open(file_name_term,'a+')
    text = concate_term_newline(list_term)
#     print(text)    
    f1.write(text)
    f1.close()


In [15]:
def concate_term_newline(dist_in):
    hasil = ''
    str_temp2 = ''
    for i in dist_in:
        for j in dist_in[i]:
            str_temp2 = str_temp2 + ","+j
            
        str_temp = i+ " : " + str_temp2+ "\n"
        str_temp2 = ""
        hasil = hasil + str_temp
        
    return hasil

# tulis data

In [16]:
def writeFile(dict_in,file_name):
    
    str_of_list = concate_term_newline(dict_in)
    
#     print(str_of_list)
    saveFile = open(file_name, 'w+' )
    saveFile.write(str_of_list)
    saveFile.close()
#     print(list_hasil)
    

<h2>Add Doc Number</h2>

In [17]:
def addDocNumber(_listIn, _docNum):
#     _docNum[14:18]
    for i in range(len(_listIn)):
        _listIn[i] = _listIn[i] + '_' + _docNum[14:18]
        
    return _listIn

<h1>Main</h1>

In [18]:
listOfFiles = readFile('DataRouter')
listOfWordsOfFile = []
counter = 1
blok = 10

dict_list_term = defaultdict(list)
blok_file_name = "term_blok"
blok_name = []

for i in listOfFiles: #iterasi tiap file
    txt = readText(i)
    
    txt = normalisasi(txt)
    listOfWords = make_token(txt)
    listOfWords = removeNull(listOfWords)
    listOfWords = caseFolding(listOfWords)
    listOfWords = removeStopWord(listOfWords)
    listOfWords = lemmatization(listOfWords)
#     listOfWords = stemming(listOfWords)
    
#     Diganti KEVIN
#     listOfWords = addDocNumber(listOfWords,i)
    
    
    for j in listOfWords:
#         if j not in list_term:
#         list_term[j].append(i) #15:01:01.79 : ,DataRouter\Doc0001.txt
        dict_list_term[j].append(i[14:18]) #15:01:01.79 : ,0001
    
#sorting ascending
dict_list_term = OrderedDict(sorted(dict_list_term.items()))

writeFile(dict_list_term,"hasil_inverted.txt")
#     counter = counter + 1
#     print(counter)
#     print(list_term)
    

In [19]:
# for i in dict_list_term:
#     print(i, ":", list_term[i])